In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd 

from matplotlib.pyplot import cm

import matplotlib.pyplot as plt
%matplotlib inline

data_path = "/mnt/c/users/geneb/documents/USC/work/casden"

/home/gene/.local/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


# Co Star Data

Lusk bought it for the state of California. Data come from Data Export under the Analytics tab (2021) for sub-markets. 

The panel goes back all the way to 2000 on a quarterly basis

Note, it's possible to download the data that we plot straight from Co-Star (at market, annual level) but I compute it here since we'll use these data for forecasts

## Load Data

In [2]:
#df = pd.read_excel("/".join([data_path, 'costar_historic_submarket.xlsx']),engine = 'openpyxl')

In [3]:
df = pd.read_excel("/".join([data_path, 'DataExport.xlsx']),engine = 'openpyxl')

In [4]:
df.columns.str.contains("Gross")

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

## Process Data

Key column names:
- Period
- Geography Name
- Market Asking Rent/SF
- Market Asking Rent/Unit
- Market Effect Rent/SF
- Vacancy Rate
- Absorption Units
- Occupancy Rate (1- vacancy rate)

**Weights** are computed as the share of total units that are in a sub-market

In [5]:
# Get market (because costar don't do that for some reason)
df.loc[:,'market'] = df['Geography Name'].str.extract("([\w\s]+) - CA -",expand=False).str.strip()

In [6]:
# extract year
df.loc[:,'year'] = df.Period.str.extract("(\d+) Q", expand=False).str.strip().astype('int')

In [7]:
# extract quarters
df.loc[:,'quarter'] = df.Period.str[-2:].str.strip()

In [8]:
# Seems I downloaded all of CA
intrs = ['Los Angeles','Orange','San Diego','Ventura','Inland Empire']
df = df.loc[df.market.str.contains("|".join(intrs)),:]

# Sub-market Level

Problem here is that we must combine markets

Consider **Los Angeles**. 
In the Casden report, there are only 11 markets while Co-Star has 36 sub-markets. Moreover, they do not, or rather cannot, be directly overlapped. I do my best to match geographies though

# Re-assign sub-market 

This is needed to agree with markets in the Casden Report

In [9]:
df.loc[:,'casden_submark'] = ''

### Los Angeles

In [10]:
df.loc[df['Geography Name'].str.contains("Los Angeles"),'Geography Name'].str.replace("Los Angeles - CA - ",'').sort_values().unique()
LA = df['Geography Name'].str.contains("Los Angeles")

In [11]:
cond = df['Geography Name'].str.contains("Glendale|Burbank|Sun Valley|Northeast Los Angeles")
df.loc[cond & LA, 'casden_submark'] = 'Burbank-Glendale'

In [12]:
cond = df['Geography Name'].str.contains("Glendale|Burbank|Sun Valley|Northeast Los Angeles")

In [13]:
df.loc[cond & LA, 'casden_submark'].unique()

array(['Burbank-Glendale'], dtype=object)

In [14]:
cond = df['Geography Name'].str.contains("Long Beach/Ports|Beach Communities|South Bay")
df.loc[cond & LA, 'casden_submark'] = 'Long Beach-South Bay'

In [15]:
cond = df['Geography Name'].str.contains("Beverly Hills/Century City/UCLA|Greater Culver City|Santa Monica|West Hollywood|West County")
df.loc[cond & LA, 'casden_submark'] = 'Coastal Communities - Beverly Hills'

In [16]:
df.casden_submark.unique()

array(['', 'Long Beach-South Bay', 'Coastal Communities - Beverly Hills',
       'Burbank-Glendale'], dtype=object)

In [17]:
cond = df['Geography Name'].str.contains("Lancaster|Palmdale|Santa Clarita")
df.loc[cond & LA, 'casden_submark'] = 'Palmdale-Lancaster-Santa Clarita'

In [18]:
cond = df['Geography Name'].str.contains("San Fernando Valley|Woodland Hills|Tarzana|Panorama|Northridge|Van Nuys|Studio")
df.loc[cond & LA, 'casden_submark'] = 'San Fernando Valley'

In [19]:
cond = df['Geography Name'].str.contains("Inglewood")
df.loc[cond & LA, 'casden_submark'] = 'Inglewood'

In [20]:
cond = df['Geography Name'].str.contains("Pasadena")
df.loc[cond & LA, 'casden_submark'] = 'Pasadena'

In [21]:
cond = df['Geography Name'].str.contains("San Gabriel")
df.loc[cond & LA, 'casden_submark'] = 'San Gabriel'

In [22]:
cond = df['Geography Name'].str.contains("South Los Angeles|Southeast Los Angeles")
df.loc[cond & LA, 'casden_submark'] = 'South/Southeast Los Angeles'

In [23]:
cond = df['Geography Name'].str.contains("Downtown Los Angeles|Westlake")
df.loc[cond & LA, 'casden_submark'] = 'Downtown'

In [24]:
cond = df['Geography Name'].str.contains("Mid Wilshire|Koreatown|North Hollywood|East Hollywood")
df.loc[cond & LA, 'casden_submark'] = 'Koreatown-Mid City'

### San Diego

Casden divides into 4, Co-Star has 14 sub-markets (including Outlying San Diego)

In [25]:
df.loc[df['Geography Name'].str.contains("San Diego"),'Geography Name'].str.replace("San Diego - CA - ",'').sort_values().unique()
SD = df['Geography Name'].str.contains("San Diego")

In [26]:
cond = df['Geography Name'].str.contains("North Shore Cities|La Jolla/UTC|Coronado/Point Loma|Balboa Park|East San Diego/El Cajon|Downtown San Diego")
df.loc[cond & SD, 'casden_submark'] = 'City of San Diego (Coastal)'

In [27]:
cond = df['Geography Name'].str.contains("Mission Valley/North Central|'National City/South Central|East San Diego/El Cajon")
df.loc[cond  & SD, 'casden_submark'] = 'City of San Diego (Inland)'

In [28]:
cond = df['Geography Name'].str.contains("Chula Vista/Imperial Beach")
df.loc[cond & SD , 'casden_submark'] = 'Chula Vista-National City'

In [29]:
cond = df['Geography Name'].str.contains("North I-15 Corridor|Poway/Santee/Ramona|North County")
df.loc[cond & SD, 'casden_submark'] = 'North County'

### Orange County

Casden divides it into 5 areas, co-star has 10 sub-markets

In [30]:
df.loc[df['Geography Name'].str.contains("Orange County"),'Geography Name'].str.replace("Orange County - CA - ",'').sort_values().unique()
OC = df['Geography Name'].str.contains("Orange County")

In [31]:
cond = df['Geography Name'].str.contains("Anaheim|Central OC West of I-5")
df.loc[cond & OC, 'casden_submark'] = 'Anaheim-West Orange Central'

In [32]:
cond = df['Geography Name'].str.contains("North County")
df.loc[cond & OC, 'casden_submark'] = 'La Habra-North OC'

In [33]:
cond = df['Geography Name'].str.contains("Central OC East of I-5|South County|Irvine|Tustin")
df.loc[cond & OC, 'casden_submark'] = 'Irvine-South-East-OC'

In [34]:
cond = df['Geography Name'].str.contains("Huntington Beach/Seal Beach|Newport Beach|Costa Mesa")
df.loc[cond & OC, 'casden_submark'] = 'Huntington-Seal-Coastal-OC'

### Inland Empire

Casden divides this into 4 areas whereas Co-Star divides it into 8 pieces. Good thing, if there's overlap then this is the best we get

In [35]:
df.loc[df['Geography Name'].str.contains("Inland Empire"),'Geography Name'].str.replace("Inland Empire - CA - ",'').sort_values().unique()
IE = df['Geography Name'].str.contains("Inland Empire")

In [36]:
cond = df['Geography Name'].str.contains("Palm Springs|Outlying San Bernardino County")
df.loc[cond & IE, 'casden_submark'] = 'Palm Springs-Indio'

In [37]:
cond = df['Geography Name'].str.contains("Riverside/Corona|San Jacinto|Southwest Riverside County/Temecula")
df.loc[cond & IE, 'casden_submark'] = 'West Riverside County'

In [38]:
cond = df['Geography Name'].str.contains("Greater Ontario/Rancho Cucamonga|San Bernardino")
df.loc[cond & IE, 'casden_submark'] = 'Rancho Cucamonga-San Bernardino'

### Ventura

Casden has 2 areas (simi and oxnard) while Co-star has 10

In [39]:
df.loc[df['Geography Name'].str.contains("Ventura"),'Geography Name'].str.replace("Ventura - CA - ",'').sort_values().unique()
Vn = df['Geography Name'].str.contains("Ventura")

In [40]:
cond = df['Geography Name'].str.contains("Port Hueneme|Oxnard|Ventura")
df.loc[cond & Vn, 'casden_submark'] = 'Oxnard-Port Hueneme-Ventura'

In [41]:
cond = df['Geography Name'].str.contains("Simi Valley|Thousand Oaks|Oak Park")
df.loc[cond & Vn, 'casden_submark'] = 'Simi Valley'

# Compute Unit-based weights

### Sub-market weights within Markets

In [42]:
# compute weights (weights are essentially share of units)
df.loc[:,'totalUnits_qrt'] = df.groupby(['Period','market'])['Demand Units'].transform('sum')
#df.loc[:,'totalUnits_yr'] = df.groupby(['year','market'])['Demand Units'].transform('sum')

# essentially the share of units that this submarket has
df.loc[:,'market_weight_qrt'] = np.divide(df['Demand Units'],df['totalUnits_qrt'])
#df.loc[:,'market_weight_yr'] = np.divide(df['Demand Units'],df['totalUnits_yr'])

### Sub-market weights within Casden-ish Sub-markets

In [43]:
cond = (df.casden_submark != '')
df.loc[:,'totalUnits_qrt_cas'] = 0
df.loc[:,'market_weight_qrt_cas'] = 0

In [44]:
# compute weights (weights are essentially share of units)
df.loc[cond,'totalUnits_qrt_cas'] = df.loc[cond,:].groupby(['Period','casden_submark'])['Demand Units'].transform('sum')
#df.loc[:,'totalUnits_yr'] = df.groupby(['year','market'])['Demand Units'].transform('sum')

# essentially the share of units that this submarket has
df.loc[cond,'market_weight_qrt_cas'] = np.divide(df['Demand Units'],df['totalUnits_qrt_cas'])
#df.loc[:,'market_weight_yr'] = np.divide(df['Demand Units'],df['totalUnits_yr'])

In [45]:
df.loc[cond,:].groupby(['Period','casden_submark']).market_weight_qrt_cas.sum()

Period   casden_submark             
2000 Q1  Anaheim-West Orange Central    1.0
         Burbank-Glendale               1.0
         Chula Vista-National City      1.0
         City of San Diego (Coastal)    1.0
         City of San Diego (Inland)     1.0
                                       ... 
2021 Q1  San Fernando Valley            1.0
         San Gabriel                    1.0
         Simi Valley                    1.0
         South/Southeast Los Angeles    1.0
         West Riverside County          1.0
Name: market_weight_qrt_cas, Length: 2040, dtype: float64

# Save the data

In [46]:
df[['Geography Name','Period','Vacancy Rate','Market Asking Rent/Unit','Under Construction Buildings']].dropna(how='any')

,Geography Name,Period,Vacancy Rate,Market Asking Rent/Unit,Under Construction Buildings
2635,Inland Empire - CA - Greater Ontario/Rancho Cu...,2000 Q1,0.030945,962.320435,2
2636,Inland Empire - CA - Greater Ontario/Rancho Cu...,2000 Q2,0.030672,983.852173,2
2637,Inland Empire - CA - Greater Ontario/Rancho Cu...,2000 Q3,0.030590,1009.071472,3
2638,Inland Empire - CA - Greater Ontario/Rancho Cu...,2000 Q4,0.030443,1027.035156,4
2639,Inland Empire - CA - Greater Ontario/Rancho Cu...,2001 Q1,0.030805,1052.648682,4
...,...,...,...,...,...
18525,Ventura - CA - Ventura,2020 Q1,0.090287,2003.302734,4
18526,Ventura - CA - Ventura,2020 Q2,0.078770,2055.125244,4
18527,Ventura - CA - Ventura,2020 Q3,0.085847,2085.887451,3
18528,Ventura - CA - Ventura,2020 Q4,0.074298,2120.484863,3


In [47]:
# remove sub-markets for which the last observation was older than 2020
df.groupby('Geography Name').Period.transform('max').min()

'2021 Q1'

In [48]:
df.to_csv("/".join([data_path, 'costart_prepped.csv']), index=False)

# Visualize

## Market-level

In [ ]:
# take inner product between rents and unitShares
#By quarter
marketAvg = {}
marketAvg['rent_quart'] =  df.groupby(['market','Period'])[['Market Asking Rent/Unit','market_weight_qrt']].apply(lambda x: np.dot(x.iloc[:,0],x.iloc[:,1])).round(-1)
marketAvg['vacancy_quart'] = df.groupby(['market','Period'])[['Vacancy Rate','market_weight_qrt']].apply(lambda x: np.dot(x.iloc[:,0],x.iloc[:,1])).round(3)

# combine into single df
marketAvg = pd.concat(marketAvg,axis=1)

# By year
#avgMarketRent_yr = df.groupby(['market','year'])[['Market Asking Rent/Unit','market_weight_yr']].apply(lambda x: np.dot(x.iloc[:,0],x.iloc[:,1])).round(-1)

# quarter average
marketAvg = marketAvg.reset_index().rename(columns = {0:'rent'})
marketAvg.loc[:,'year'] = marketAvg.Period.str.extract("(\d+) Q", expand=False).str.strip().astype('int').reset_index(drop=True)

In [ ]:
toplot = marketAvg.groupby(['market','year'])[['rent_quart','vacancy_quart']].mean()

In [ ]:
toplot

# Visualize Each Market

### average rents

In [ ]:
for label, group in toplot['rent_quart'].groupby(level=0):
    plt.figure()
    group[label].plot(kind = 'bar',width=.93,color='orange')
    plt.title("Average rents in %s" %label, fontsize = 14)
    plt.xlabel("Year", fontsize = 14)
    plt.ylabel("Avg. asking rent per unit", fontsize = 14)
    plt.savefig("/".join([data_path, 'avg_annual_rent_%s.png' %label]))

### Vacancy Rates

In [ ]:
for label, group in toplot['vacancy_quart'].groupby(level=0):
    plt.figure()
    group[label].plot(kind = 'bar',width=.93,color='teal')
    plt.title("Average rents in %s" %label, fontsize = 14)
    plt.xlabel("Year", fontsize = 14)
    plt.ylabel("Avg. asking rent per unit", fontsize = 14)
    plt.savefig("/".join([data_path, 'avg_annual_vacancy_%s.png' %label]))

## Compute averages

In [ ]:
# Compute average rents
avg = df.groupby(['Geography Name','year'])['Market Asking Rent/Unit'].mean().round(-1)

In [ ]:
avg = avg.reset_index()

In [ ]:
avg['Geography Name'].unique().shape[0]

In [ ]:
avg.loc[avg['Geography Name'].str.contains("Los Angeles"),'Geography Name'].unique().shape[0]